<a href="https://colab.research.google.com/github/Sujithra-dhayalan/MiniProject/blob/main/SepsiPre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# Load dataset
df_train = pd.read_csv('/content/Paitients_Files_Train.csv')

# Encode the target variable
df_train['prediction'] = df_train['Sepssis'].map({'Positive': 1, 'Negative': 0})

# Drop unnecessary columns
df_train = df_train.drop(['ID', 'Sepssis'], axis='columns')

# Separate features and target
x = df_train.drop('prediction', axis='columns')
y = df_train['prediction']

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# Feature scaling for models that need it
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=100000, class_weight=class_weight_dict),
    'Random Forest': RandomForestClassifier(class_weight=class_weight_dict),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier()  # For handling class imbalance
}

# Evaluate models
model_names = []
model_avg_score = []

# Cross-validation using F1 score or ROC AUC (better for imbalanced data)
for model_name, model in models.items():
    scores = cross_val_score(model, x_train_scaled, y_train, scoring='roc_auc', cv=5)


    model_names.append(model_name)
    model_avg_score.append(scores.mean())

# Output the results
df_mode = pd.DataFrame()
df_mode['Model Name'] = model_names
df_mode['Average Score'] = model_avg_score
df_mode


,Model Name,Average Score
0,Logistic Regression,0.829486
1,Random Forest,0.817839
2,Gradient Boosting,0.812873
3,XGBoost,0.784745
